Lab02-2
======

###  Context
#### Regression(Linear Regression)
   + Feature selection(Step wise linear regression)
   + Decision Tree Regression 

## 데이터 살펴보기

+ 이번 실습에서는 scikit-learn에서 제공하는 예제 데이터 중 하나인 당뇨병 환자 데이터를 사용합니다.
<br>데이터를 살펴보기 쉽게 csv 파일을 pandas DataFrame으로 읽습니다.

In [ ]:
from os.path import join
import numpy as np

from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score
from sklearn.tree import export_graphviz

import matplotlib.pyplot as plt

import statsmodels.api as sm

import pandas as pd
from pandas.plotting import scatter_matrix

In [ ]:
df = pd.read_csv(join('data','diabetesDataset.csv'))
print(df.shape)

In [ ]:
df.head()

##  \* 회귀 분석은 단순히 회귀식을 추정하는 것만을 의미하지 않습니다.

+ 추정된 회귀식이 정말로 종속 변수와 독립 변수간 변랑을 잘 설명할 수 있는지, 추정된 회귀 계수가 정말로 의미 있는 값인지에 대한 검정을 해야 합니다.

### \- 추정된 회귀 계수가 정말로 의미 있는 값인가?

+ 지금까지는 독립 변수들끼리 모두 독립임을 가정한 상태로 분석을 진행하였습니다.


+ 하지만 만약 두 변수가 서로에게 영향을 주고 있다면 문제가 발생합니다.


    예를 들어 학업 성취도를 종속 변수 Y로 놓고 이를 설명해주는 독립 변수 X1은 일평균 음주량을 또다른 독립 변수 X2는 혈중 알코올 농도라고 하겠습니다. 이때 일평균 음주량이나 혈중 알코올 농도가 높을수록 학업 성취도가 낮아질 것으로 추측할 수 있습니다.
    하지만 일평균 음주량과 혈중 알코올 농도는 서로 완전히 독립적이지 않습니다.
    이런 상태에서 두 변수를 사용하여 회귀 분석을 하였을 경우 변수들 각각의 설명력은 약해지게 됩니다.
    
### \- 다중 공선성 (multicollinearity)
+ 독립 변수의 일부가 다른 독립 변수의 조합으로 표현될 수 있는 경우를 말합니다.
<br>(회귀 분석에서 사용된 모형의 일부 독립 변수가 다른 독립 변수와 상관 정도가 높아, 데이터 분석 시 부정적인 영향을 미치는 현상)

## 다중 공선성 문제
+ 독립 변수와 종속 변수 간에 유의한 관계가 존재하는 경우에도 계수가 유의하지 않은 것으로 보일 수 있습니다

+ 높은 상관 관계가 있는 독립 변수에 대한 계수는 표본에 따라 크게 달라질 수 있습니다.

+ 높은 상관 관계가 있는 독립 변수 항을 제거하면 높은 상관 관계가 있는 다른 독립 변수 항의 추정 계수에 크게 영향을 미칩니다.

## 이를 방지하는 방법
+ 변수 선택법(Feature Selection)으로 의존적인 변수 삭제하는 방법
+ PCA(principal component analysis) 방법으로 의존적인 성분 삭제하는 방법
+ 정규화(regularized) 방법


# \* 이번 시간에는 가장 기본적인 방법인 다른 독립변수에 의존하는 변수를 없애는 방법에 대해 실습하고 이를 바탕으로 독립변수를 선택하는 실습을 하겠습니다.

## 상관계수를 통한 1차적 진단

corr() 메서드는 모든 특성 간의 상관계수(피어슨의 표준 상관계수)를 행렬로 줍니다. 

\- 상관관계의 범위는 -1부터 1까지 입니다.

\- 1에 가까우면 강한 양의 상관관계, -1에 가까우면 강한 음의 상관관계를 가진다는 뜻입니다.


\- 상관계수가 0에 가까우면 선형적인 상관관계가 없다는 뜻입니다.


In [ ]:
df.corr()

In [ ]:
dfx = df.iloc[:,:-1]
dfy = df.iloc[:,-1]
# df = pd.concat([dfx,dfy],axis = 1)

scatter matrix

In [ ]:
dfx1 = df.loc[:,'age':'bmi']
dfx2 = df.loc[:,'map':'ldl']
dfx3 = df.loc[:,'hdl':'glu']

dfy = df.loc[:,'y']

df1 = pd.concat([dfx1,dfy], axis = 1)
df2 = pd.concat([dfx2,dfy], axis = 1)
df3 = pd.concat([dfx3,dfy], axis = 1)


scatter_matrix(df1, figsize = (18,12))
plt.show()

In [ ]:
scatter_matrix(df2, figsize = (18,12))
plt.show()

In [ ]:
scatter_matrix(df3, figsize = (18,12))
plt.show()

- 산점도 행렬이나 상관관계수를 계한하여 상관 관계가 높은 독립변수들을 판단하고 다중공선성 문제가 일어날 것이라는 예상을 할 수 있습니다만 아닙니다.

- 상관관계는 인과관계를 의미하진 않습니다.
<br>:x와 y가 강한 상관관계를 보인다면 x가 y를 발생시켰다고 볼 수도 있고, y가 x를 발생시켰다고 볼 수도 있습니다. 혹은 서로 동시에 발생시키거나, 다른 외부 요인이 발생시키거나 아니면 아무런 인과관계도 없을 수 있습니다.

## \* 시각적으로 독립변수 간의 상관 관계를 시각적으로 판단할 수 있으나, 관계 정도가 유의한지 알아보기 볼 필요가 있습니다.
### - 유의성 검정

## < Stepwise Linear Regression >

## Feature selection 
+ 전방 탐색(Forward Search)
    1. 0개 변수에서 시작합니다.
    2. 한번에 하나의 변수에 대해서 시도합니다.
    3. 회귀 모델을 만들고 각 변수의 기여도를 측정합니다.
    4. 가장 높은 기여도를 가진 변수를 추가합니다.
    5. 선택하지 않은 변수 중에 통계적으로 중요한 변수가 남지 않을 때 까지 반복합니다.
    
    
+ 후방 소거(Backward Eliminate)
    1. 모든 변수를 가지고 시작합니다.
    2. 한번에 하나의 변수에 대해서 제거를 시도합니다.
    3. 회귀 모델을 만들고 각 변수의 기여도를 측정합니다.
    4. 가장 낮은 기여도를 가진 변수를 삭제합니다.
    5. 통계적으로 중요하게 제거할 변수가 남지 않을 때 까지 반복합니다.
    
    
+ 단계적 선택(Stepwise Selection)
    <br>전방 탐색 + 후방 소거
    <br>전방 탐색과 비슷하게 변수를 추가합니다.
    <br>단계에 상관 없이 가장 중요하지 않은 변수를 제거하도록 합니다.

## p-value 분석을 통한 변수 선택(Feature Selection)
+ 다중공선성이 생기면 해당 변수의 설명력은 약해집니다.
+ 이는 변수의 표준오차의 증가로 드러납니다.
+ 검정 통계량 = (추정된 회귀 계수 - H0) / 그 계수의 추정 표준 오차
+ 이 검정 통계량의 절대값이 클수록 p-value는 작아져서(표준 오차가 커졌기 때문) 귀무가설을 기각할 수 있게 됩니다.
+ 가설 검정을 통해서 판단합니다.
    + 귀무가설을 회귀 계수 = 0이라고 설정
    + 그 회귀 계수가 유의미하다고 판단하면 기각
    


1)여기서 __표준 오차는 표본의 실제관측치가 표본 회귀선상의 추정치에서 얼마나 흩어져 있나를 측정한 값입니다.__

2) 표준오차가 작으면 작을수록 실제값과 추정값간의 차이가 없다고 볼수 있습니다.

3) 표준오차 구하기 
### $s = \sqrt{\sum \frac{(Y_i-\hat{Y})^2}{n-2}}$

(추정표준오차의 제곱($s^2$)은 잔차의 분산입니다.)

ex) Y = 2(X1) + 3(X2) <다중 공선성 문제가 있는 X1, X2>

=> X1이 설명력 있는 독립 변수라면 p-value가 유의 수준보다 작아 유의한 통계량이 됩니다.

(문제 발생)그런데, X2가 설명할 부분을 X1이 가져가 버렸기 때문에 X2의 설명력은 작아지게 됩니다.

![stepwise](./images/stepwise.png)

## < Stepwise Linear Regression >

In [ ]:
# 컬럼이름으로 다중 컬럼 선택 (return DatFrame)
## 하나의 컬럼을 선택할 경우 Series를 반환
## 하나의 컬럼을 DataFrame으로 반환하기 위해서는 컬럼 리스트로 입력

label = 'bmi'
diabetes_X = df.iloc[:,:-1]
diabetes_Y = df[['y']]

In [ ]:
print(diabetes_X.shape)
diabetes_X.head()

In [ ]:
diabetes_Y.head()

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in= 0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(diabetes_X, diabetes_Y)

print('resulting features:')
print(result)

newlabel = result

In [ ]:
diabetes_X_multi = df[newlabel].values.tolist()
diabetes_Y_multi = df[['y']].values.tolist()

# 선형회귀 추정기 생성
lr = linear_model.LinearRegression() 

# input 대해 선형 회귀(모델 파라미터 추정)
lr.fit(diabetes_X_multi, diabetes_Y_multi)

# 회귀식으로 데이터의 결과 추정
diabetes_y_pred_multi = lr.predict(diabetes_X_multi)

# R2 scroe
R2 = r2_score(diabetes_Y_multi, diabetes_y_pred_multi)
print('R2 score: %.2f' % R2)

# adj R2 score
n = len(diabetes_X_multi)
p = len(newlabel)

Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
print('Adj R2 score: %.2f' % Adj_r2)

In [ ]:
import itertools

newlabel = ['age', 'sex', 'bmi', 'map', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']
combs = []
adj_list = []
label_list = []
for i in range(1, len(newlabel)+1):
    els = [list(x) for x in itertools.combinations(newlabel, i)]
    combs.extend(els) # list 원소만을 append(일반적인 append는 list 자체를 추가한다.)
for i in range(len(combs)):
    print('label', combs[i])
    label_list.append(combs[i])
    dfx = df[combs[i]].values.tolist()
    dfy = df[['y']].values.tolist()
    print(np.shape(dfx))

    # 선형회귀 추정기 생성
    lr2 = linear_model.LinearRegression()

    # input 대해 선형 회귀(모델 파라미터 추정)
    lr2.fit(dfx, dfy)

    # 모델 파라미터 출력
    print('Model parameters: \n')
    rgstr = ''
    for j in range(len(lr2.coef_[0])):
        print("b%d" %(len(lr2.coef_[0])-j),": ", "%f" %lr2.coef_[0,j])
        rgstr = ' + '+repr(lr2.coef_[0,j]) + '*x'+repr(len(lr2.coef_[0])-j) + rgstr

    print('b0:', lr2.intercept_[0])
    print()
    rgstr = repr(lr2.intercept_[0]) + rgstr
    print('y = ', rgstr)

    y_pred = lr2.predict(dfx)
    n = len(dfx)
    p = len(combs[i])

    R2 = r2_score(dfy, y_pred)
    Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
    adj_list.append(Adj_r2)
    print('Adj R2 score: %.2f' %  Adj_r2)
    print('**************************************************************')

In [ ]:
print('Max Adj R2 score: %.2f' %  np.max(adj_list))
print('Selected label :', label_list[np.argmax(adj_list)])

## Decision Tree Regression 

+ 회귀를 위한 의사결정트리 학습에서는 정보이득의 __불순도 측정을 평균 제곱 오차__로 합니다.

의사결정트리의 학습에서 각 노드에서 분기하기 위한 최적의 질문은 정보이득이라는 값을 최대가 되도록 만듭니다.

어느 특정 노드에서 m개의 자식 노드로 분기되는 경우 정보이득은 아래의 식으로 정의합니다.

### 정보이득 : $IG(D_p,f) = I(D_p) - \sum_{j=1}^{m}{\frac{N_j}{N_p} I(D_j)}$

$ I(D_p)$: $D_p$의 데이터 불순도
<br>$I(D_j)$: $D_j$의 데이터 불순도
<br>$N_p$: $D_p$의 데이터 개수
<br>$N_j$: $D_j$의 데이터 개수

데이터 불순도 : 데이터가 제대로 분류되지 않고 섞여 있는 정도를 말합니다.

정보이득 $IG$는 자식노드의 데이터 불순도가 작으면 작을수록 커지게 됩니다.

In [ ]:
label = ['bmi']
diabetes_X = df[label]
diabetes_Y = df[['y']]

다음은 의사결정트리 회귀를 위한 DecisionTreeRegressor 예측기 입니다.

In [ ]:
# 결정트리 회귀 모델 생성
tr = DecisionTreeRegressor(max_depth=2)

fit() 메서드를 통해 모델 파라미터를 추정합니다.

In [ ]:
tr.fit(diabetes_X,diabetes_Y)

### \- result

훈련된 결정트리 시각화

In [ ]:
# with open("TreeRegr.dot", "w") as f:
#     f = export_graphviz(tr, out_file=f)

In [ ]:
# 'Graphviz'라는 오픈소스 그래프 시각화 소프트웨어 패키지를 사용하여 그래프를 시각화할 수 있습니다.

# 파이썬 패키지, graphviz 설치
# conda isntall python-graphviz

# dot to png
# !dot -Tpng TreeRegr.dot -o TreeRegr.png

max depth = 3a
![TreeRegr](./images/TreeRegr.png)

회귀의 결과를 그래프로 표시하기 위해 X의 요소값이 작은 순서로 정렬한 것의 인덱스를 sort_idx로 둡니다.
<br>ex) [1,0,5].argsort() => [1,0,2]

In [ ]:
sort_idx = diabetes_X.values.ravel().argsort()

In [ ]:
plt.scatter(diabetes_X.iloc[sort_idx],diabetes_Y.iloc[sort_idx],c = 'lightgray', label ='Training Data')
plt.plot(diabetes_X.iloc[sort_idx], tr.predict(diabetes_X.iloc[sort_idx]))
# , c = 'red', label = 'Tree, $R^2 = %.2f$' %r2_score(diabetes_Y, tr.predict(diabetes_X[sort_idx])))
plt.xlabel('bmi')
plt.ylabel('y')
plt.legend()
plt.show()

### \- evaluation

In [ ]:
print('R2 score: %.2f' % r2_score(diabetes_Y, tr.predict(diabetes_X)))